### Introduction
This notebook illustrates and automates the Continuous Deployment process for bringing the popular open-source large language models service [Ollama](https://ollama.com//) into SAP AI Core. Running with Llama 2, Mistral, Gemma, and other large language models in SAP AI Core through BYOM(Bring Your Own Model) approach. <br/>
### Prerequisites
Before running this notebook, please assure you have perform the [Prerequisites](../../README.md)<br/>
 
### The high-level flow of this Continuous Deployment process:
- Build a custom Ollama docker image adapted for SAP AI Core<br/>
- Push the docker image to docker hub<br/>
- Connect to SAP AI Core via SDK<br/>
- Create a deployment<br/>
- Check the status and logs of the deployment<br/>


#### 1.Build a custom Ollama docker image adapted for SAP AI Core
Please refer to [Dockerfile](Dockerfile) for details.

In [7]:
%%sh
# 0.Login to docker hub
docker login -u csritej269 -p dckr_pat_adWAMIiSrJWvKVDDrpCdynvKYYs docker.io

# 1.Build the docker image
docker build --platform=linux/amd64 -t docker.io/csritej269/ollama:ai-core .

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


Login Succeeded


#0 building with "desktop-linux" instance using docker driver

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 2.14kB done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:22.04
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:22.04
#3 DONE 1.9s

#5 [1/5] FROM docker.io/library/ubuntu:22.04@sha256:a6d2b38300ce017add71440577d5b0a90460d0e57fd7aec21dd0d1b0761bbfb2
#5 DONE 0.0s

#6 [2/5] RUN apt-get update &&     apt-get install -y     ca-certificates     nginx     curl &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
#6 CACHED

#7 [3/5] RUN curl -fsSL https://ollama.com/install.sh | sh
#7 CACHED

#8 [4/5] RUN echo "events { use epoll; worker_connections 128; }     http {         server {                     listen 8080;                         location ^~ /v1/a

#### 2.Push the docker image to docker hub

In [8]:
%%sh
# 2.Push the docker image to docker hub to be used by deployment in SAP AI Core
docker push docker.io/csritej269/ollama:ai-core 

The push refers to repository [docker.io/csritej269/ollama]
e1aec6c8c9a1: Preparing
83aba39f6721: Preparing
300eec0f015d: Preparing
bdac9835c688: Preparing
629ca62fb7c7: Preparing
83aba39f6721: Pushed
e1aec6c8c9a1: Pushed
bdac9835c688: Pushed
629ca62fb7c7: Pushed


#### 3.Initiate an SAP AI Core SDK client
- resource_group loaded from [../config.json](../config.json)
- ai_core_sk(service key) loaded from [../config.json](../config.json)

In [ ]:
import requests, json, time, datetime
from datetime import datetime
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

In [ ]:
# load the configuration from ../config.json 
with open("../config.json") as f:
    config = json.load(f)

resource_group = config.get("resource_group", "default")
print( "resource group: ", resource_group)

resource group:  oss-llm


In [ ]:
# Initiate an AI Core SDK client with the information of service key
ai_core_sk = config["ai_core_service_key"]
base_url = ai_core_sk.get("serviceurls").get("AI_API_URL") + "/v2/lm"
client = AICoreV2Client(base_url=ai_core_sk.get("serviceurls").get("AI_API_URL")+"/v2",
                        auth_url=ai_core_sk.get("url")+"/oauth/token",
                        client_id=ai_core_sk.get("clientid"),
                        client_secret=ai_core_sk.get("clientsecret"),
                        resource_group=resource_group)

In [ ]:
# Prepare the http header which will be used later through request.
token = client.rest_client.get_token()
headers = {
    "Authorization": token,
    "ai-resource-group": resource_group,
    "Content-Type": "application/json",
}

#### 4.Create a deployment for Ollama scenario
To create a deployment in SAP AI Core, it requires the corresponding resource_group and configuration_id
- resource_group loaded from [../config.json](../config.json)
- configuration_id of  loaded from [env.json](env.json)

In [ ]:
# resource_group: The target resource group to create the deployment
# configuration_id: The target configuration to create the deployment, which is created in ../00-init-config.ipynb 
with open("./env.json") as f:
    env = json.load(f)

configuration_id = env["configuration_id"]
print("configuration id:", configuration_id)

configuration id: 2567b2ae-b5d1-4214-92e9-ae28e523174e


**Helper function**
- get the current UTC time in yyyy-mm-dd hh:mm:ss format, to be used to filter deployments logs

In [6]:
# Helper function to get the current time in UTC, used to filter deployments logs
def get_current_time():  
    current_time = datetime.utcnow()
    # Format current time in the desired format
    formatted_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    return formatted_time

**Helper function**
- Write back the configuration value back to configuration json file

In [7]:
# Helper function to write back the configuration value back to configuration json file
def update_json_file(file_path, key, value):
    # Load the JSON configuration file
    with open(file_path, 'r') as file:
        config = json.load(file)

    # Update the value
    config[key] = value

    # Write the updated configuration back to the file
    with open(file_path, 'w') as file:
        json.dump(config, file, indent=4)
        print(f"{file_path} updated. {key}: {value}")

**Create a deployment for Ollama in SAP AI Core**
- configuration_id
- resource_group
<br/><br/>
The created deployment id will be written back to [env.json](env.json), which will be used in
- [02-ollama.ipynb](02-ollama.ipynb) and [03-ollama-llava.ipynb](03-ollama-llava.ipynb) to test the inference of open-source llms with Ollama in SAP AI Core
- [04-cleanup.ipynb](04-cleanup.ipynb) to stop the deployment and clean up the resource.

In [8]:
# Create a Deployment in SAP AI Core
print("Creating deployment.")
response = client.deployment.create(
    configuration_id=configuration_id,
    resource_group=resource_group
)

# last_check_time will be used to check the deployment status continuously afterwards
# set initial last_check_time right after creating deployment
last_check_time = get_current_time()
deployment_start_time = datetime.now()

deployment_id = response.id
status = response.status
update_json_file("env.json", "deployment_id", deployment_id)
print("Deployment Result:\n", response.__dict__)

Creating deployment.
env.json updated. deployment_id: da55eac132ecd29d
Deployment Result:
 {'id': 'da55eac132ecd29d', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


#### 5.Check the status and logs of the deployment

In [12]:
print("4.Checking deployment status.")
deployment_url = f"{base_url}/deployments/{deployment_id}"
deployment_log_url = f"{deployment_url}/logs?start="
interval_s = 20

while status != "RUNNING" and status != "DEAD":
    current_time = get_current_time()
    #check deployment status
    response = requests.get(url=deployment_url, headers=headers)
    resp = response.json()
    
    status = resp['status']
    print(f'...... Deployment Status at {current_time}......', flush=False)
    print(f"Deployment status: {status}")

    #retrieve deployment logs
    response_log = requests.get(url=f"{deployment_log_url}{last_check_time}", headers=headers)
    last_check_time = current_time
    print(f"Deployment logs: {response_log.text}")

    # Sleep for 60 secs to avoid overwhelming the API with requests
    time.sleep(interval_s)

deployment_end_time = datetime.now()
duration_in_min = (deployment_end_time - deployment_start_time) / 60

if status == "RUNNING":
    print("Deployment is up and running now!")
else:
    print(f"Deployment {deployment_id} failed!")   

print(f"Deployment duration: {duration_in_min} mins")

4.Checking deployment status.
...... Deployment Status at 2024-05-05T11:11:12.572273Z......
Deployment status: UNKNOWN
Deployment logs: {
  "error": {
    "code": "05011000",
    "message": "DeploymentNotFoundError: Deployment da55eac132ecd29d not found.",
    "target": "/api/v4alpha/deployments/da55eac132ecd29d/logs"
  }
}

...... Deployment Status at 2024-05-05T11:11:34.097148Z......
Deployment status: UNKNOWN
Deployment logs: {
  "error": {
    "code": "05011000",
    "message": "DeploymentNotFoundError: Deployment da55eac132ecd29d not found.",
    "target": "/api/v4alpha/deployments/da55eac132ecd29d/logs"
  }
}

...... Deployment Status at 2024-05-05T11:11:55.717299Z......
Deployment status: UNKNOWN
Deployment logs: {
  "error": {
    "code": "05011000",
    "message": "DeploymentNotFoundError: Deployment da55eac132ecd29d not found.",
    "target": "/api/v4alpha/deployments/da55eac132ecd29d/logs"
  }
}

...... Deployment Status at 2024-05-05T11:12:17.365084Z......
Deployment status

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))